In [1]:
import cv2
import os
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.neighbors import NearestNeighbors
import numpy as np
import bz2, gzip, pickle

In [2]:
%load_ext line_profiler

In [3]:
index_im_fol = os.path.abspath("Images/index")
index_images_nl = [x for x in os.listdir(index_im_fol)]
index_images = [os.path.join(index_im_fol, x) for x in os.listdir(index_im_fol)]

In [4]:
index_descriptors = {}
surf = cv2.xfeatures2d.SURF_create(1000)

In [5]:
# def multi_descrip(idx, image):
#     surf = cv2.xfeatures2d.SURF_create(10000)
#     global index_descriptors
#     img = cv2.imread(image)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     kp, des = surf.detectAndCompute(img,None)
#     if des is not None:
#         return (idx,dex)

In [6]:
# def parallelize_multi_descrip(kargs):
#     multi_descrip(*kargs)

In [7]:
def pickle_desc(i_desc, count):
    with gzip.open('index_descrip/descriptor_dictionary_' + str(count) + '.pickle', 'wb') as file:
        pickle.dump(i_desc, file, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
count = 0
def descrip():
    global index_descriptors, count
    for idx, image in enumerate(index_images):
        img = cv2.imread(image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = surf.detectAndCompute(img,None)
        if des is not None:
            index_descriptors[index_images_nl[idx]] = des
        if idx % 1000 == 0 and idx != 0:
            pickle_desc(index_descriptors, count)
            print('Saved: ', count)
            count += 1
            del index_descriptors
            index_descriptors = {}
        del img
        del kp
        del des

In [9]:
descrip()
# 70s

Saved:  0
Saved:  1
Saved:  2
Saved:  3
Saved:  4
Saved:  5
Saved:  6
Saved:  7
Saved:  8
Saved:  9
Saved:  10
Saved:  11
Saved:  12
Saved:  13
Saved:  14
Saved:  15
Saved:  16
Saved:  17
Saved:  18
Saved:  19
Saved:  20
Saved:  21
Saved:  22
Saved:  23
Saved:  24
Saved:  25
Saved:  26
Saved:  27
Saved:  28
Saved:  29
Saved:  30
Saved:  31
Saved:  32
Saved:  33
Saved:  34
Saved:  35
Saved:  36
Saved:  37
Saved:  38
Saved:  39
Saved:  40
Saved:  41
Saved:  42
Saved:  43
Saved:  44
Saved:  45
Saved:  46
Saved:  47
Saved:  48
Saved:  49
Saved:  50
Saved:  51
Saved:  52
Saved:  53
Saved:  54
Saved:  55
Saved:  56
Saved:  57
Saved:  58
Saved:  59
Saved:  60
Saved:  61
Saved:  62
Saved:  63
Saved:  64
Saved:  65
Saved:  66
Saved:  67
Saved:  68
Saved:  69
Saved:  70
Saved:  71
Saved:  72
Saved:  73
Saved:  74
Saved:  75
Saved:  76
Saved:  77
Saved:  78
Saved:  79
Saved:  80
Saved:  81
Saved:  82
Saved:  83
Saved:  84
Saved:  85
Saved:  86
Saved:  87
Saved:  88
Saved:  89
Saved:  90
Saved:  9

KeyboardInterrupt: 

In [12]:
descrip_vec = None
for des in index_descriptors.values():
    if descrip_vec is None:
        descrip_vec = des
    else:
        descrip_vec = np.concatenate((descrip_vec, des), axis=0)

In [13]:
descrip_vec.shape

(136, 64)

In [14]:
with gzip.open('descrip_vec.pickle', 'wb') as file:
    pickle.dump(index_descriptors, file, protocol=pickle.HIGHEST_PROTOCOL)

In [49]:
km = None
def cluster():
    global km
    km = KMeans(n_clusters = 1000).fit(descrip_vec)

The documentation suggests to try MiniBatchKMeans for large data, much faster than regular KMeans

In [ ]:
%lprun -f cluster cluster()
#  11.962s

In [ ]:
nbrs = NearestNeighbors(n_neighbors=1).fit(km.cluster_centers_)
hist_dict = {}
for img in index_descriptors:
    dist, idx = nbrs.kneighbors(index_descriptors[img])
    hist = np.zeros((1, 1000))
    for i in idx:
        hist[:,i-1] += 1
    hist_dict[img] = hist

In [ ]:
print(1)